# 🐱🐶 Projeto de Transfer Learning - Cats vs Dogs
Este notebook demonstra como aplicar **Transfer Learning** utilizando o modelo **MobileNetV2 pré-treinado no ImageNet** para classificar imagens de **Gatos vs Cães**.

📌 Autor: Sérgio Santos  
📌 Dataset: [Cats vs Dogs (TensorFlow Datasets)](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs)

---

In [ ]:
# =======================================================
# 1. Instalar e importar dependências
# =======================================================
!pip install tensorflow tensorflow-datasets matplotlib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import os, json

## 🔧 Configurações gerais

In [ ]:
IMG_SIZE = (160, 160)
BATCH_SIZE = 32
EPOCHS = 5
MODEL_DIR = "models"
HISTORY_DIR = "history"

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(HISTORY_DIR, exist_ok=True)

## 📥 2. Carregar e preparar dataset
O dataset **Cats vs Dogs** será carregado a partir do TensorFlow Datasets. Faremos pré-processamento:
- Redimensionamento das imagens
- Normalização dos pixels (0-1)
- Divisão em treino e teste

In [ ]:
print("Carregando dataset Cats vs Dogs...")
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)

train_ds = dataset['train']
test_ds = dataset['train'].take(2000)  # parte do dataset para teste

def format_example(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0  # normalização
    return image, label

train_ds = train_ds.map(format_example).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(format_example).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

## 🧠 3. Construção do modelo com MobileNetV2
Utilizaremos o **MobileNetV2** pré-treinado no **ImageNet**, removendo a camada final e adicionando:
- `GlobalAveragePooling2D`
- `Dropout`
- `Dense(1, sigmoid)` para classificação binária

In [ ]:
print("Carregando MobileNetV2 pré-treinada...")
base_model = keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs, name="cats_vs_dogs_transfer_learning")
model.summary()

## ⚙️ 4. Compilar e configurar callbacks
- Otimizador **Adam** com learning rate reduzido
- Loss: `binary_crossentropy`
- Métricas: `accuracy`, `precision`, `recall`
- Callbacks: `ModelCheckpoint` e `EarlyStopping`

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

checkpoint_cb = callbacks.ModelCheckpoint(
    filepath=os.path.join(MODEL_DIR, "transfer_learning_best.keras"),
    save_best_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=1
)

earlystop_cb = callbacks.EarlyStopping(
    patience=3,
    restore_best_weights=True,
    monitor="val_accuracy"
)

## 🚀 5. Treinamento do modelo

In [ ]:
print("Treinando modelo...")
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_cb, earlystop_cb]
)

## 📊 6. Avaliar desempenho

In [ ]:
loss, acc, prec, rec = model.evaluate(test_ds)
print(f"\nDesempenho no conjunto de teste:")
print(f"  Loss: {loss:.4f}")
print(f"  Acurácia: {acc:.4f}")
print(f"  Precisão: {prec:.4f}")
print(f"  Recall: {rec:.4f}")

## 💾 7. Salvar modelo e histórico

In [ ]:
final_model_path = os.path.join(MODEL_DIR, "transfer_learning_final.keras")
model.save(final_model_path)
print(f"Modelo salvo em: {final_model_path}")

history_path = os.path.join(HISTORY_DIR, "training_history.json")
with open(history_path, "w") as f:
    json.dump(history.history, f)
print(f"Histórico salvo em: {history_path}")

## 📈 8. Curvas de aprendizado

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Treino Acurácia')
plt.plot(history.history['val_accuracy'], label='Validação Acurácia')
plt.xlabel("Épocas")
plt.ylabel("Acurácia")
plt.legend()
plt.grid(True)
plt.title("Curva de Aprendizado - Transfer Learning")
plt.savefig(os.path.join(HISTORY_DIR, "accuracy_curve.png"))
plt.show()